<a href="https://colab.research.google.com/github/RiverGumSecurity/AILabs/blob/dev-derek/labs/Module02_Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Supervised Learning and Neural Networks with Phishing Email Data

Neural networks are a subset of machine learning algorithms inspired by the human brain's structure and function. They consist of layers of interconnected "neurons" that transform the input data through weighted connections to produce an output. Neural networks are particularly powerful for complex tasks like image recognition and natural language processing.

Neural networks consist of an input layer, one or more hidden layers, and an output layer. These layers consist of "neurons" where each neuron processes input data by applying a weight and an activation function (like sigmoid or ReLU) to produce an output. The training (or learning) process adjusts the weights of the connections between neurons using algorithms like backpropagation and optimization techniques like gradient descent. Neural networks, particularly deep neural networks (deep learning), can model highly complex relationships between inputs and outputs by learning hierarchical feature representations in the hidden layers. Deep learning neural networks typically contain three or more hidden layers.

<center><img src="https://images.edrawsoft.com/articles/neural-network-diagram/example1.png" width="800" height="600"></center>




In [ ]:
# import libraries
import numpy as np
import pandas as pd
import nltk
import seaborn as sns
import plotly.express as px
import datetime
import os 
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score,f1_score,classification_report,ConfusionMatrixDisplay,confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding,GRU,LSTM,Bidirectional,SimpleRNN
from tensorflow.keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Dense,Dropout
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Supress debug warnings in training output
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Suppresses INFO and WARNING messages
tf.get_logger().setLevel('ERROR')

In [ ]:
# Set random seed static value for consistent results
seed = 42

In [ ]:
# Detect GPU device
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Suppresses INFO and WARNING messages

## Dataset

The dataset that will be used to train and test the different models was from https://www.kaggle.com/datasets/subhajournal/phishingemails.  It was pre-labeled as "Safe Email" or "Phishing Email".  The data will be cleaned and prepared for training machine learning models with Pandas.

In [ ]:
# Read in the data into a Pandas dataframe
df = pd.read_csv('https://raw.githubusercontent.com/RiverGumSecurity/Datasets/main/Kaggle/Phishing_Email.csv.gz')
#df = pd.read_csv('../datasets/Phishing_Email.csv')
df.head(10)

In [ ]:
# Print information on the dataframe
df.info()

In [ ]:
# Drop null values in place, drop Unamed:0 column, drop duplicates
df.drop(['Unnamed: 0'],axis=1,inplace=True)
df.dropna(inplace=True,axis=0)
df.drop_duplicates(inplace=True)

In [ ]:
# Show the shape of the data
df.shape

In [ ]:
# Display cleaned dataframe
df.head(10)

In [ ]:
# Display the count of Email types, a Safe Email or a Phishing Email
df['Email Type'].value_counts()

In [ ]:
# Plot counts of Safe Email vs Phishing Email
df['Email Type'].value_counts().plot(kind='bar')

Now we need to further process and clean the data. This is a binary classification problem, and we need to assign a label (a 1 or a 0) to the email categories. Then we will remove URLS and non word characters from the emails - we are interested in the similarities of the text itself.  Then we lowercase all of the characters, convert all multiple whitespace characters to single whitespace, and remove any trailing whitespaces.

In [ ]:
# Label Email Type
lbl = LabelEncoder()
df['Email Type'] = lbl.fit_transform(df['Email Type'])

In [ ]:
# Function to preprocess text.
def preprocess_text(text):
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()
    text = re.sub(r'\s+', ' ', text).strip()
    return text
df['Email Text']=df['Email Text'].apply(preprocess_text)

In [ ]:
df.head(10)

## Simple Neural Network

The [`SimpleRNN`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/SimpleRNN) layer in TensorFlow is a [recurrent neural network (RNN)]9https://en.wikipedia.org/wiki/Recurrent_neural_network) layer that processes sequential data, making it suitable for tasks like time series prediction or text generation. Each unit in SimpleRNN maintains a hidden state across time steps, allowing the model to "remember" previous inputs in the sequence. It supports configurations for returning only the final output or the entire sequence, and can also be stacked with other layers for deeper architectures. This layer is simpler and faster than more complex RNN types, like LSTM or GRU, but may struggle with longer sequences.

Because our email messages are variable length, we need to fix the length of each vector we create. In deep learning, it’s often beneficial to have inputs of consistent shapes. Setting max_len ensures that all sequences are padded or truncated to a specific length, allowing you to batch-process sequences more efficiently. 


In [ ]:
#Set max_len
max_len = 256

We initalize a new [Tokenizer](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer) class which is used for preprocessing text data by converting words or characters into numerical representations that a machine learning model can understand. In general, a token is a word or a character.  In our case, a token is a word.

In [ ]:
#Initialize new instance of Tokenizer class.
tk = Tokenizer()

The next step is to use `fit_on_texts` method in the 'Tokenizer' class to  processes each text entry in the `Email Text` column of the Pandas data frame to build a vocabulary, where each unique word (or token) is assigned a unique integer index based on its frequency across the entire dataset. It then creates a word index mapping each word (token) to a unique integer in each email. 

Then we use the `texts_to_sequences` method to create a list of integer sequence for each email from our word index mappings.  These sequence of integers are then converted to vectors of fixed length (256 from our max_len variable) and padded or truncated as necessary and stored in a numpy array.


In [ ]:
# Process the text
tk.fit_on_texts(df['Email Text'])
sequences = tk.texts_to_sequences(df['Email Text'])
vector = pad_sequences(sequences,padding='post', maxlen=max_len)

In [ ]:
#Number of rows in the vector numpy array, matches the number of emails in our dataset.
vector.shape[0]

In [ ]:
#Length of our word index
len(tk.word_index) + 1

Now we set the `max_features` parameter to limit our overall vocabulary size to the most frequent words. There are two main reasons for this:

- Reduces Memory Usage: In many NLP tasks, datasets contain thousands or millions of unique words. Limiting the vocabulary size to the most frequent words helps reduce memory and computational requirements, which is especially helpful when working with limited resources or large datasets.
- Improves Model Generalization: High vocabulary sizes can lead to overfitting, as the model might memorize rare words that do not generalize well to new data. Setting max_features forces the model to focus on the most important and commonly occurring words, which generally improves performance on unseen data

In [ ]:
max_features=10000

Now we split our dataset into training and testing sets using the `train_test_split function` from scikit-learn.  We will train our neural network on 80 percent of the data, then evaluate the accuracy on the remaining 20 percent. 

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(vector,df['Email Type'], test_size=0.2, random_state =0)

In [ ]:
model_srnn = Sequential() # Sequential() API
model_srnn.add(Embedding(len(tk.word_index)+1,50))
model_srnn.add(SimpleRNN(units=100))
model_srnn.add(Dropout(0.5))
model_srnn .add(Dense(1, activation='sigmoid'))

In [ ]:
model_srnn.compile(loss='binary_crossentropy' , optimizer='adam', metrics=['accuracy'])

In [ ]:
historical_srnn  = model_srnn.fit(x_train,y_train, epochs=20, batch_size=16, validation_data=(x_test,y_test))

In [ ]:

pd.DataFrame(historical_srnn.history)

pd.DataFrame(historical_srnn.history)[['accuracy', 'val_accuracy']].plot()
plt.title('Training Accuracy')
plt.xlabel('Epochs')
plt.ylabel('accuracy')

pd.DataFrame(historical_srnn.history)[['loss', 'val_loss']].plot()
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')

In [ ]:
y_pred_prob_smp = model_srnn.predict(x_test)
y_pred_smp = (y_pred_prob_smp > 0.5).astype(int)


cnf_smp = confusion_matrix(y_test,y_pred_smp)
ax_smp = ConfusionMatrixDisplay(confusion_matrix=cnf_smp,display_labels=['phishing','normal']).plot()
plt.title("Confusion Matrix")
plt.show()

In [ ]:
# Evaluate the model on the test set
srnn_loss, srnn_accu = model_srnn.evaluate(x_test, y_test, verbose=0)

srnn_accu = (srnn_accu*100)

# Display the accuracy value
print(f'The accuracy of the SRNN model on the test data set was: {srnn_accu:.4f}')

## LTSM

In [ ]:
model_lstm = Sequential() # Sequential() API
model_lstm.add(Embedding(len(tk.word_index)+1,50))
model_lstm.add(LSTM(units=100))
model_lstm.add(Dropout(0.5))
model_lstm.add(Dense(1, activation='sigmoid'))

In [ ]:
model_lstm.compile(loss='binary_crossentropy' , optimizer='adam', metrics=['accuracy'])

In [ ]:
historical_lstm  = model_lstm.fit(x_train,y_train, epochs=20, batch_size=16, validation_data=(x_test,y_test))

In [ ]:
lstm_results = model_lstm.evaluate(x_test, y_test)
lstm_loss = lstm_results[0]  # Extract the loss from the results
lstm_accu = (lstm_results[1]*100)  # Extract the accuracy from the results

print(f"Model Loss: {lstm_loss}")
print(f"Model Accuracy: {lstm_accu}")

In [ ]:
y_pred_prob = model_lstm.predict(x_test)
y_pred = (y_pred_prob > 0.5).astype(int)

In [ ]:
pd.DataFrame(historical_lstm.history)

pd.DataFrame(historical_lstm.history)[['accuracy', 'val_accuracy']].plot()
plt.title('Training Accuracy')
plt.xlabel('Epochs')
plt.ylabel('accuracy')

pd.DataFrame(historical_lstm.history)[['loss', 'val_loss']].plot()
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')

In [ ]:
cnf = confusion_matrix(y_test,y_pred)
ax = ConfusionMatrixDisplay(confusion_matrix=cnf,display_labels=['pishing','normal'])
ax.plot()
plt.title("Confusion Matrix")
plt.show()

## Bidirectional

In [ ]:
model_bi = Sequential() # Sequential() API
model_bi.add(Embedding(len(tk.word_index)+1,50))
model_bi.add(Bidirectional(LSTM(units=100)))
model_bi.add(Dropout(0.5))
model_bi.add(Dense(1, activation='sigmoid'))

In [ ]:
model_bi.compile(loss='binary_crossentropy' , optimizer='adam', metrics=['accuracy'])

In [ ]:
historical_bi  = model_bi.fit(x_train,y_train, epochs=20, batch_size=16, validation_data=(x_test,y_test))

In [ ]:
bi_results = model_bi.evaluate(x_test, y_test)
bi_loss = bi_results[0]  # Extract the loss from the results
bi_accu = (bi_results[1]*100)  # Extract the accuracy from the results

print(f"Model Loss: {bi_loss}")
print(f"Model Accuracy: {bi_accu}")

In [ ]:
y_pred_prob_bi = model_bi.predict(x_test)
y_pred_bi = (y_pred_prob_bi > 0.5).astype(int)

In [ ]:
pd.DataFrame(historical_bi.history)

pd.DataFrame(historical_bi.history)[['accuracy', 'val_accuracy']].plot()
plt.title('Training Accuracy')
plt.xlabel('Epochs')
plt.ylabel('accuracy')

pd.DataFrame(historical_bi.history)[['loss', 'val_loss']].plot()
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')

In [ ]:
cnf_bi = confusion_matrix(y_test,y_pred_bi)
ax_bi = ConfusionMatrixDisplay(confusion_matrix=cnf_bi,display_labels=['Phishing','Normal'])
ax_bi.plot()
plt.show()

### Gated Recurrent Unit

In [ ]:
model_gru = Sequential() # Sequential() API
model_gru.add(Embedding(len(tk.word_index)+1,50))
model_gru.add(GRU(units=100))
model_gru.add(Dropout(0.5))
model_gru.add(Dense(1, activation='sigmoid'))

In [ ]:
model_gru.compile(loss='binary_crossentropy' , optimizer='adam', metrics=['accuracy'])

In [ ]:
historical_gru = model_gru.fit(x_train,y_train, epochs=20, batch_size=16, validation_data=(x_test,y_test))

In [ ]:
gru_results = model_gru.evaluate(x_test, y_test)
gru_loss = gru_results[0]  # Extract the loss from the results
gru_accu = (gru_results[1]*100)  # Extract the accuracy from the results

print(f"Model Loss: {gru_loss}")
print(f"Model Accuracy: {gru_accu}")

In [ ]:
y_pred_prob_gru = model_gru.predict(x_test)
y_pred_gru = (y_pred_prob_gru > 0.5).astype(int)

In [ ]:
pd.DataFrame(historical_gru.history)

pd.DataFrame(historical_gru.history)[['accuracy', 'val_accuracy']].plot()
plt.title('Training Accuracy')
plt.xlabel('Epochs')
plt.ylabel('accuracy')

pd.DataFrame(historical_gru.history)[['loss', 'val_loss']].plot()
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')

In [ ]:
cnf_gru = confusion_matrix(y_test,y_pred_gru)
ax_gru = ConfusionMatrixDisplay(confusion_matrix=cnf_gru,display_labels=['Pishing','normal'])
ax_gru.plot()
plt.show()

# Model Comparisons

In [ ]:
# Need to do - set up table of models and comparisons

accu_values = [srnn_accu,lstm_accu, bi_accu, gru_accu]
row_labels = ["Simple RNN", "LSTM Neural Network", "Bidirectional Neural Network", "Gated Recurrent Unit"]
comp_df = pd.DataFrame(accu_values, columns=["accuracy"], index=row_labels)
print(comp_df)